### Define helper functionality to visualize forward and inverse image mapping

In [1]:
from pathlib import Path

import Metashape

from mynd.backend import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

import os

from mynd.collections import CameraGroup, StereoCameraGroup
from mynd.backend import metashape as backend
from mynd.geometry import load_hitnet
from mynd.utils.environment import Environment, load_environment

from mynd.tasks.export_stereo import (
    ExportStereoTask,
    invoke_stereo_export_task,
)

GroupID = CameraGroup.Identifier


def prepare_task_config(
    identifier: GroupID, stereo_camera: StereoCameraGroup
) -> ExportStereoTask.Config:
    """Prepares a stereo export task config by creating setting up paths, creating directories,
    and loading a disparity model."""

    environment: Environment = load_environment().unwrap()

    model: HitnetConfig = load_hitnet(
        environment.resource_directory
        / Path("hitnet_models/hitnet_eth3d_720x1280.onnx")
    ).unwrap()

    # Root directories
    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_ranges")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_normals")

    # NOTE: Debug only!
    logger.info(f"Camera pairs: {len(stereo_camera.camera_pairs)}")
    logger.info(f"Image loaders: {len(stereo_camera.image_loaders)}")

    config = ExportStereoTask.Config(
        range_directory=RANGE_DIRECTORY / Path(f"{identifier.label}_ranges"),
        normal_directory=NORMAL_DIRECTORY / Path(f"{identifier.label}_normals"),
        model=model,
        calibrations=stereo_camera.calibrations,
        camera_pairs=stereo_camera.camera_pairs,
        image_loaders=stereo_camera.image_loaders,
    )

    # Create export directories if they do not exist
    if not config.range_directory.exists():
        os.mkdir(config.range_directory)
    if not config.normal_directory.exists():
        os.mkdir(config.normal_directory)

    return config


def main():
    """Main function."""

    DOCUMENT_PATH: Path = Path(
        "/data/kingston_snv_01/acfr_metashape_projects_dev/r23m7ms0_lite_with_metadata.psz"
    )

    match backend.load_project(DOCUMENT_PATH):
        case Ok(None):
            pass
        case Err(message):
            logger.error(message)

    identifiers: list[GroupID] = backend.get_group_identifiers().unwrap()

    target: GroupID = identifiers[0]

    match backend.camera_services.retrieve_stereo_cameras(target):
        case Ok(stereo_camera_groups):
            for stereo_cameras in stereo_camera_groups:
                config: ExportStereoTask.Config = prepare_task_config(
                    target, stereo_cameras
                )
                result: Result[ExportStereoTask.Result, str] = (
                    invoke_stereo_export_task(config)
                )
        case Err(message):
            logger.error(message)


# ---------- Invoke main function ----------
main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


LoadProject: path = /data/kingston_snv_01/acfr_metashape_projects_dev/r23m7ms0_lite_with_metadata.psz
loaded project in 18.6325 sec


2024-10-20 11:46:27.997425653 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 20 Memcpy nodes are added to the graph tf2onnx for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-10-20 11:46:28.066 | INFO     | __main__:prepare_task_config:41 - Camera pairs: 2569
2024-10-20 11:46:28.066 | INFO     | __main__:prepare_task_config:42 - Image loaders: 5138
Estimating stereo geometry...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2569/2569 [26:50<00:00,  1.60it/s]
